# This file is used to train

In [1]:
import torch
import numpy as np
import pandas as pd
from sc_model import SC_LSTM as Model
from tg_bot.domain.domain import Task, Event
import datetime

# Device init

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    !nvidia-smi
    print(torch.cuda.get_device_name(0))

else:
    print("No GPU :(")

Tue Jul 11 05:33:54 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti      Off| 00000000:01:00.0  On |                  N/A |
| N/A   43C    P5                9W /  N/A|    814MiB /  4096MiB |     33%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
device = 'cpu'

## Loading the data

In [4]:
resched_tasks = [
        Task(telegram_id=0,
             task_name="task_0",
             importance=2,
             start_time=datetime.time(1, 30),
             duration=100,
             date=datetime.datetime.now().date()),
    Task(telegram_id=0,
             task_name="task_1",
             importance=2,
             start_time=datetime.time(10, 20),
             duration=20,
             date=datetime.datetime.now().date()),

    Task(telegram_id=0,
            task_name="Task_2",
            importance=3,
            start_time=datetime.time(13, 30),
            duration=60,
            date=datetime.datetime.now().date()),
    Task(telegram_id=0,
            task_name="Task_3",
            importance=1,
            start_time=datetime.time(15, 30),
            duration=30,
            date=datetime.datetime.now().date()),
    Task(telegram_id=0,
            task_name="Task_4",
            importance=1,
            start_time=datetime.time(16, 30),
            duration=120,
            date=datetime.datetime.now().date())]

## Init the model

In [5]:
in_features = 22
out_features = 3
batch_size = 1
n_layers = 1
hidden_size = 124
hidden_injector = 64

free_time_slots = [[0, 1]]

# Create the model
SC_LSTM = Model(in_features, n_layers, hidden_size, out_features, batch_size, hidden_injector=hidden_injector).to(device)

# Load the weights
SC_LSTM.load_state_dict(torch.load('sc_lstm_weights.pth'))

<All keys matched successfully>

In [6]:
from Data_Test.Preprocessor import Preprocessor

preproc = Preprocessor(alpha=1440, beta=300, num_lables=4, max_importance=5)


# Test the model on input data
SC_LSTM.reset_states()
SC_LSTM.eval_model()
predictions = []
real = []

for task in resched_tasks:
    X = preproc.preprocess_data('resched', task, 1)
    X = torch.Tensor([X]).type(torch.float32).to(device)
    # print(X)
    Y_pred = SC_LSTM.forward(X, task_type='resched', free_time_slots=free_time_slots, save_states=True)
    # print(f"Predicted: {Y_pred}, ---- Real: {X[0][13], X[0][4], 0}")

    # print prediction not in E notation
    np.set_printoptions(suppress=True)
    predictions.append(Y_pred[0].numpy())
    real.append([X[0][13], X[0][4], 0])
    print(f"Real: {X[0][13], X[0][4]} ------- Predicted: {Y_pred[0].numpy()}")


Real: (tensor(0.0625), tensor(0.3333)) ------- Predicted: [0.11528745 0.37316942 0.00015443]
Real: (tensor(0.4306), tensor(0.0667)) ------- Predicted: [0.93125    0.09588564 0.00000249]
Real: (tensor(0.5625), tensor(0.2000)) ------- Predicted: [0.9966372  0.06430619 0.00016303]
Real: (tensor(0.6458), tensor(0.1000)) ------- Predicted: [0.93125    0.11990117 0.00000173]
Real: (tensor(0.6875), tensor(0.4000)) ------- Predicted: [0.9982998  0.21354744 0.00007144]


/tmp/ipykernel_224737/1254715381.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  X = torch.Tensor([X]).type(torch.float32).to(device)


In [7]:


for i in range(0, len(input_data)):
    X = torch.Tensor([input_data[i]]).type(torch.float32).to(device)
    Y = torch.Tensor([output_data[i]]).type(torch.float32).to(device)

    Y_pred = SC_LSTM.forward(X, task_type='resched', free_time_slots=free_time_slots, save_states=True)

    predictions.append(Y_pred[0].numpy())
    real.append(Y[0].numpy())
    print(f"{i+1}) Predicted: {Y_pred[0]}\n Real: {Y[0]}\n Error: {torch.abs(Y_pred[0] - Y[0])}\n")

# Print mean, min, and max error for each column
errors = np.abs(np.array(predictions) - np.array(real))
print(f"Mean error: {np.mean(errors, axis=0)}")
print(f"Min error: {np.min(errors, axis=0)}")
print(f"Max error: {np.max(errors, axis=0)}")


# Mean error: [0.03400678 0.05854695 0.00183747]
# Min error: [0.00568134 0.01077583 0.00074498]
# Max error: [0.07480101 0.23154303 0.00484255]

NameError: name 'input_data' is not defined

In [ ]:
# Plot the barchart of an errors
import matplotlib.pyplot as plt
import numpy as np

labels = ['Start Time', 'Duration', 'Importance']
mean_errors = np.mean(errors, axis=0)
min_errors = np.min(errors, axis=0)
max_errors = np.max(errors, axis=0)

x = np.arange(len(labels))  # the label locations
width = 0.25  # the width of the bars

fig, ax = plt.subplots(figsize=(10, 5))
rects1 = ax.bar(x - width, mean_errors, width, label='Mean error')
rects2 = ax.bar(x, min_errors, width, label='Min error')
rects3 = ax.bar(x + width, max_errors, width, label='Max error')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Error')
ax.set_title('Mean, min, and max error for each column')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

ax.set_ylim([0, 0.5])

fig.tight_layout()

plt.show()